In [ ]:
%run ./../data/load-dataset.ipynb
%run ./../various/_epoch-callback.ipynb
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, RNN, GRU, LSTM, Dense, Dropout

In [ ]:
ITERS = 10

RANDOM_SEED = 0

In [ ]:
corpus, labels = df[[proc_doc_col, label_col]].T.values
X = [dictionary.doc2idx(doc) for doc in corpus]
y = np.array([np.eye(2)[int(label)] for label in labels])
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, random_state=RANDOM_SEED)
train_tensor, test_tensor = tf.ragged.constant(train_samples), tf.ragged.constant(test_samples)

In [ ]:
model = Sequential()
model.add(Input(shape=(None, ), ragged=True, dtype=tf.int64))
model.add(Embedding(input_dim=len(dictionary), output_dim=100))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(2, activation=tf.nn.sigmoid))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

training_set_hist = KerasEpochCallback(end_func=model.evaluate,
                                       end_args=(train_tensor, train_labels), 
                                       end_kwargs={'verbose': False})
test_set_hist = KerasEpochCallback(end_func=model.evaluate, 
                                   end_args=(test_tensor, test_labels), 
                                   end_kwargs={'verbose': False})
model.summary()

In [ ]:
training_hist = model.fit(train_tensor, train_labels, epochs=ITERS, batch_size=32,
                          callbacks=[training_set_hist, test_set_hist])

training_metrics = np.array(list(zip(training_hist.history['loss'], training_hist.history['accuracy'])))
training_set_metrics = np.array(training_set_hist.end_results)
test_set_metrics = np.array(test_set_hist.end_results)

results = [{'Training Loss': training_loss, 'Training Accuracy': training_acc,
            'Training Set Loss': training_set_loss, 'Training Set Accuracy': training_set_acc,
            'Test Set Loss': test_set_loss, 'Test Set Accuracy': test_set_acc} 
           for (training_loss, training_acc), 
               (training_set_loss, training_set_acc), 
               (test_set_loss, test_set_acc) 
           in zip(training_metrics, training_set_metrics, test_set_metrics)]

results_df = pd.DataFrame(results)
results_df.index += 1
results_df[['Training Loss', 'Training Accuracy', 
            'Training Set Loss', 'Training Set Accuracy', 
            'Test Set Loss', 'Test Set Accuracy']].style \
                .highlight_min(subset=['Training Accuracy', 'Training Set Accuracy', 'Test Set Accuracy'],
                               color='lightcoral') \
                .highlight_max(subset=['Training Accuracy', 'Training Set Accuracy', 'Test Set Accuracy'],
                               color='lightgreen') \
                .highlight_min(subset=['Training Loss', 'Training Set Loss', 'Test Set Loss'], 
                               color='lightgreen') \
                .highlight_max(subset=['Training Loss', 'Training Set Loss', 'Test Set Loss'], 
                               color='lightcoral')

In [ ]:
ax = plt.gca()
ax.plot(np.arange(0, ITERS), training_metrics.T[0], '-', label="During Training")
ax.plot(np.arange(0, ITERS), training_set_metrics.T[0], '--', label="On Training Set")
ax.plot(np.arange(0, ITERS), test_set_metrics.T[0], ':', label="On Test Set")
ax.legend()
plt.title("Loss")
plt.tight_layout()

In [ ]:
ax = plt.gca()
ax.plot(np.arange(0, ITERS), training_metrics.T[1], '-', label="During Training")
ax.plot(np.arange(0, ITERS), training_set_metrics.T[1], '--', label="On Training Set")
ax.plot(np.arange(0, ITERS), test_set_metrics.T[1], ':', label="On Test Set")
ax.legend()
plt.title("Accuracy")
plt.tight_layout()

In [ ]:
test_preds = model.predict(test_tensor)
true_preds, true_labels = [np.argmax(pred) for pred in test_preds], \
                          [np.argmax(label) for label in test_labels]
ConfusionMatrixDisplay.from_predictions(true_labels, true_preds, normalize='true',
                                        cmap=plt.cm.Blues, display_labels=('reliable', 'unreliable'))